# Modelo Para Previsão de Doenças Usando Registros Médicos Eletrônicos - Parte 2

<h2>Primeira Versão do Modelo</h2>

Agora que temos os dados devidamente processados, limpos e transformados. É hora de aplicar técnicas de Machine Learning com o objetivo de encontrar o padrão nos dados históricos para conseguir prever o futuro com base nesses padrões.

Esta parte 2 tem um caráter mais introdutório aos dados estudados. Vamos aplicar alguns algoritmos sem muitos critérios. O objetivo é realizar uma primeira modelagem e a partir disso tentar melhorar com técnicas mais avançadas.

Irei aplicar três técnicas:
* **KNN**: Vamos verificar se a relação por proximidade de vizinhança se mostra boa o suficiente;
* **Regressão Logística**: Um método linear generalizado. A regressão logística é útil para modelar a probabilidade de um evento ocorrer em função de outros fatores.
* **Random Forest**: E também irei utilizar a técnicas de árvores de decisão.

<h3>Importações das bibliotecas</h3>

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pandas as pd

#from imblearn.under_sampling import RandomUnderSampler

<h3>Carrega os Dados Pré-Processados</h3>

Vamos carregar os dados já transformados e padronizados que foram salvos no notebook anterior diretamente do bucket S3.

In [8]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)

# Altere para o nome do seu bucket
s3_bucket = 'krupck-bucket-bloodpressure'
prefix = 'dados'

arn:aws:iam::351371806175:role/service-role/AmazonSageMaker-ExecutionRole-20220722T092670


In [9]:
raiz = 's3://{}/{}/'.format(s3_bucket, prefix)
print(raiz)

s3://krupck-bucket-bloodpressure/dados/


In [10]:
#Dados de treino
df_treino = pd.read_csv(raiz + 'treino.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
display(df_treino.head())

#Dados de teste
df_teste = pd.read_csv(raiz + 'teste.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
display(df_teste.head())

,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
0,0,-0.745894,-1.483066,-1.557970,-1.230564
1,1,1.103977,-1.483066,0.484622,0.839808
2,0,-1.938942,-1.483066,-1.557970,-0.777549
3,0,1.315880,-0.490421,1.228612,0.752458
4,0,0.112748,0.708359,-0.744431,1.523915


,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
0,0,0.454357,-0.085644,0.057265,-0.118441
1,0,-0.664451,0.413848,0.484622,0.909034
2,1,-1.075770,-1.483066,-1.557970,0.576501
3,0,0.314809,1.162859,-0.744431,0.962562
4,0,1.561199,-0.490421,0.019759,-1.230564


**Carregando os dados de maneira local**

In [ ]:
#Dados de treino
df_treino = pd.read_csv('dados/treino.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
display(df_treino.head())

#Dados de teste
df_teste = pd.read_csv('dados/teste.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
display(df_teste.head())

<h4>Separando variáveis preditoras da variável target</h4>

In [ ]:
X_treino = df_treino[['bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate']].values
y_treino = df_treino['class']

X_teste = df_teste[['bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate']]
y_teste = df_teste['class']

y_treino

<h4>Balanceando o dataset</h4>

Como o dataset está bem desbalanceado, isso pode prejudicar nas previsões do modelo, portanto, iremos aplicar técnicas para balancear o dataset com o objetivo de deixá-lo balanceado e dessa forma não prejudicar o modelo.

In [ ]:
y_treino.value_counts()

In [ ]:
rus = RandomUnderSampler()
X_res, y_res = rus.fit_resample(X_treino, y_treino)

In [ ]:
y_res.value_counts()

---
<h2>Primeira modalagem</h2>

<h3>KNN - K Nearest Neighbors</h3>

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_res, y_res)

preds = knn.predict(X_teste)
f_score = f1_score(preds, y_teste)
ac_score = accuracy_score(preds, y_teste)

print('F1 score em Teste = {:.2f}%'.format(f_score * 100))
print('Acurácia em Teste = {:.2f}%'.format(ac_score * 100))

<h3>Logistic Regression</h3>

In [ ]:
logisticReg = LogisticRegression()
logisticReg.fit(X_res, y_res)

preds = logisticReg.predict(X_teste)
f_score = f1_score(preds, y_teste)
ac_score = accuracy_score(preds, y_teste)

print('F1 score em Teste = {:.2f}%'.format(f_score * 100))
print('Acurácia em Teste = {:.2f}%'.format(ac_score * 100))

<h3>Random Forest</h3>

In [ ]:
randomForest = RandomForestClassifier()
randomForest.fit(X_res, y_res)

preds = randomForest.predict(X_teste)
f_score = f1_score(preds, y_teste)
ac_score = accuracy_score(preds, y_teste)

print('F1 score em Teste = {:.2f}%'.format(f_score * 100))
print('Acurácia em Teste = {:.2f}%'.format(ac_score * 100))

---
<h2>Considerações Finais</h2>

Vemos que a precisão dos modelos foram horríveis em todos os casos. Se não tivessimos balanceado o dataset, a acurácia teria sido de mais de 80%. Quais são as conclusões até aqui:
* Balancear o dataset foi muito importante para entender como os dados realmente se comportam;
* Esses dados estão enviesados, após todos os estudos eu chego a conclusão de que ou os dados foram simulados ou o dataset não condizem com a realidade. Baseado em que:
    1. A maioria das pessoas possuiam magreza extrema;
    2. A relação entre todas as categorias de atributos estavam proporcionalmente balanceados, e isso é bem difícil de acontecer na realidade;
    3. Mesmo balanceando o dataset, o modelo não conseguiu aprender a relação entre os dados;

---
<h2>Continua...</h2>

O objetivo nos próximos notebooks será estudar o Amazon SageMaker.
* Vamos criar e treinar um modelo no ambiente em nuvem;
* Vamos gerar um endpoint para realizar previsões a partir do modelo criado;